# Estimation du coefficent de ruissellment #

## Import des bibliothèques ##

In [1]:
from prettytable import PrettyTable
import numpy as np
from scipy.stats import linregress
import pandas as pd

## Définition des fonctions ##

In [50]:
class DataAnalyzer:
    def __init__(self, rows):
        self.rows = rows

    def logarithmic_interpolation(self, x, y, target):
        log_x = np.log(x)
        log_y = np.log(y)
        log_target = np.log(target)
        y_expanded = np.repeat(log_y, len(log_x) // len(log_y))
        slope, intercept, _, _, _ = linregress(log_x, y_expanded)
        interpolated_log_value = slope * log_target + intercept
        interpolated_value = np.exp(interpolated_log_value)
        return interpolated_value

    def interpolate_values(self):
        for i in range(len(self.rows)):
            if "Conditions" not in self.rows[i][0]:
                for j in range(2, len(self.rows[i])):
                    if self.rows[i][j] == "":
                        if self.rows[i][j-1] and self.rows[i][j+1]:
                            left_val = float(self.rows[i][j-1])
                            right_val = float(self.rows[i][j+1])
                            interpolated_value = self.logarithmic_interpolation([2, 5, 10, 25, 50, 100], [left_val, right_val], 30)
                            self.rows[i][j] = str(round(interpolated_value, 2))
        # Ajouter le code pour imprimer la table ici
        table_chow = PrettyTable()
        table_chow.field_names = ["Type de surface", "Pente", "T = 2 ans", "T = 5 ans", "T = 10 ans", "T = 25", "T = 30 ans", "T = 50 ans", "T = 100 ans", "T = 500 ans"]
        table_chow.max_width["Type de surface"] = 20
        table_chow.max_width["Pente"] = 15
        table_chow.max_width["T = 2 ans"] = 10
        table_chow.max_width["T = 5 ans"] = 10
        table_chow.max_width["T = 10 ans"] = 10
        table_chow.max_width["T = 25"] = 10
        table_chow.max_width["T = 30 ans"] = 10
        table_chow.max_width["T = 50 ans"] = 10
        table_chow.max_width["T = 100 ans"] = 10
        table_chow.max_width["T = 500 ans"] = 10
        for row in self.rows:
            table_chow.add_row(row)

        # Afficher la table
        print(table_chow)

class BasinInfo:
    def __init__(self):
        self.surface_types = ["Surface Asphalte (ha)", "Surface Béton,Toit (ha)", "Surface Zones enherbées (ha)",
                              "Surface Zone cultivée (ha)", "Surface Pâturage, Prairie (ha)", "Surface Fôret (ha)"]

    def create_table(self):
        num_basins = int(input("Entrer le nombre des bassins : "))
        if num_basins <= 0:
            print("Nombre de bassins invalide. Veuillez entrer un entier positif.")
            return

        basin_info = []
        for i in range(num_basins):
            name = input(f"Le nom du bassin versant {i+1}: ")
            if not name:
                print("Veuillez renseigner les noms de vos bassins versants")
                return
            slope = input(f"La pente du basin '{name}' en m/m: ")
            if not slope:
                print("Veuillez renseigner les pentes de vos bassins versants")
                return
            surface_values = []
            for surface in self.surface_types:
                value = input(f"La {surface} du basin '{name}': ")
                surface_values.append(value)
            basin_info.append([name, slope] + surface_values)

        table_entrants = PrettyTable()
        table_entrants.field_names = ["Nom du basin versant", "Pente"] + self.surface_types
        for basin in basin_info:
            table_entrants.add_row(basin)
        print(table_entrants)
        return basin_info

    def calculer_coefficients_ruissellement(self, basin_info):
        # Extraction des informations de surface à partir des données de bassin
        num_basins = len(basin_info)
        basin_names = [info[0] for info in basin_info]
        slopes = [float(info[1]) for info in basin_info]
        asphalt = [float(info[2]) for info in basin_info]
        beton = [float(info[3]) for info in basin_info]
        zone_enherbee = [float(info[4]) for info in basin_info]
        zone_cultivee = [float(info[5]) for info in basin_info]
        paturage = [float(info[6]) for info in basin_info]
        foret = [float(info[7]) for info in basin_info]
        
        # Initialisation des listes pour stocker les coefficients pour chaque période de retour
        coefficients = {2: [], 5: [], 10: [], 25: [], 30:[], 50: [], 100: []}

        # Calcul des coefficients pour chaque période de retour
        for T_index in range(2, 9): 
            if T_index == 2:
                T_value = 2
            elif T_index == 3:
                T_value = 5
            elif T_index == 4:
                T_value = 10
            elif T_index == 5:
                T_value = 25
            elif T_index == 6:
                T_value = 30
            elif T_index == 7:
                T_value = 50
            elif T_index == 8:
                T_value = 100

            for i in range(num_basins):
                surface_total = sum([asphalt[i], beton[i], zone_enherbee[i], zone_cultivee[i], paturage[i], foret[i]])
                zone_enherbee_a = 0
                zone_enherbee_b = 0
                zone_enherbee_c = 0
            
            
                if zone_enherbee[i] / surface_total < 0.5:
                    zone_enherbee_a = zone_enherbee[i]
                elif 0.5 <= zone_enherbee[i] / surface_total < 0.75:
                    zone_enherbee_b = zone_enherbee[i]
                elif zone_enherbee[i] / surface_total >= 0.75:
                    zone_enherbee_c = zone_enherbee[i]
            
                if slopes[i] <= 0.02:
                    Cr = round((asphalt[i] * float(rows[0][T_index]) + beton[i] * float(rows[1][T_index]) + zone_enherbee_a * float(rows[4][T_index]) + zone_enherbee_b * float(rows[8][T_index]) +
                            zone_enherbee_c * float(rows[12][T_index]) + zone_cultivee[i] * float(rows[16][T_index]) + paturage[i] * float(rows[20][T_index]) + foret[i] * float(rows[24][T_index])) / surface_total, 2)
                elif 0.02 < slopes[i] < 0.07:
                    Cr = round((asphalt[i] * float(rows[0][T_index]) + beton[i] * float(rows[1][T_index]) + zone_enherbee_a * float(rows[5][T_index]) + zone_enherbee_b * float(rows[9][T_index]) +
                            zone_enherbee_c * float(rows[13][T_index]) + zone_cultivee[i] * float(rows[17][T_index]) + paturage[i] * float(rows[21][T_index]) + foret[i] * float(rows[25][T_index])) / surface_total, 2)
                elif slopes[i] >= 0.07:
                    Cr = round((asphalt[i] * float(rows[0][T_index]) + beton[i] * float(rows[1][T_index]) + zone_enherbee_a * float(rows[6][T_index]) + zone_enherbee_b * float(rows[10][T_index]) +
                            zone_enherbee_c * float(rows[14][T_index]) + zone_cultivee[i] * float(rows[18][T_index]) + paturage[i] * float(rows[22][T_index]) + foret[i] * float(rows[26][T_index])) / surface_total, 2)
            
                coefficients[T_value].append(Cr) 
        return coefficients
        pass

class TableCreator:
    def __init__(self, basin_info, coefficients):
        self.basin_info = basin_info
        self.coefficients = coefficients
        self.surface_types = ["Surface Asphalte (ha)", "Surface Béton,Toit (ha)", "Surface Zones enherbées (ha)",
                              "Surface Zone cultivée (ha)", "Surface Pâturage, Prairie (ha)", "Surface Fôret (ha)"]

    def create_table(self):
        basin_names = []
        total_area = []
        slopes = []
        surface_impermeabilise = []
        surface_naturelle = []

        for info in self.basin_info:
            basin_names.append(info[0])
            numeric_values = [float(val) if val.replace('.', '', 1).isdigit() else 0 for val in info[1:]]
            total_area.append(sum(numeric_values))
            slopes.append(numeric_values[0])
            surface_impermeabilise.append(sum(numeric_values[1:3]))
            surface_naturelle.append(sum(numeric_values[3:]))

        table = PrettyTable()
        table.field_names = ['Nom du bassin versant', 'Surface totale (ha)', 'Pente (m/m)',
                             'Surface imperméabilisée (ha)', 'Surface naturelle (ha)']
        
        # Set max width for each column
        table.max_width['Nom du bassin versant'] = 20
        table.max_width['Surface totale (ha)'] = 15
        table.max_width['Pente (m/m)'] = 15
        table.max_width['Surface imperméabilisée (ha)'] = 25
        table.max_width['Surface naturelle (ha)'] = 20

        for info in zip(basin_names, total_area, slopes, surface_impermeabilise, surface_naturelle):
            table.add_row(info)

        # Add coefficients to the table
        for T, cr_values in self.coefficients.items():
            table.add_column(f'CR (T={T} years)', cr_values)

        return table


## Table Ven Te chow - Coefficient de ruissellement (Applied Hydrology p 498 ) ##

In [22]:
rows = [
    ["Asphalte", "-", "0.73", "0.77", "0.81", "0.86", "", "0.9", "0.95", "1"],
    ["Béton,toit", "-", "0.75", "0.8", "0.83", "0.88", "", "0.92", "0.97", "1"],
    ["Zones enherbées (pelouse, parc…)", "", "", "", "", "", "", "", "", ""],
    ["Conditions faibles (surface enherbée < 50% de la surface totale)", "", "", "", "", "", "", "", "", ""],
    ["","Plat 0-2%", "0.32", "0.34", "0.37", "0.4", "", "0.44", "0.47", "0.58" ],
    ["","Moyen 2-7%", "0.37", "0.4", "0.43", "0.46", "", "0.49", "0.53", "0.61"],
    ["","Raide >7%", "0.4", "0.43", "0.45", "0.49", "", "0.52", "0.55", "0.62"],
    ["Conditions passables (surfaces enherbée entre 50% et 75% de la surface totale)", "","", "", "", "", "", "", "", ""],
    ["","Plat 0-2%", "0.25", "0.28", "0.3", "0.34", "", "0.37", "0.41", "0.53"],
    ["","Moyen 2-7%", "0.33", "0.36", "0.38", "0.42", "", "0.45", "0.49", "0.58"],
    ["","Raide >7%", "0.37", "0.4", "0.42", "0.46", "", "0.49", "0.53", "0.6"],
    ["Conditions bonnes (surfaces enherbée > 75% de la surface totale)", "", "", "", "", "","", "", "", ""],
    ["","Plat 0-2%", "0.21", "0.23", "0.25", "0.29", "", "0.32", "0.36", "0.49"],
    ["","Moyen 2-7%", "0.29", "0.32", "0.35", "0.39", "", "0.42", "0.46", "0.56"],
    ["","Raide >7%", "0.34", "0.37", "0.4", "0.44", "", "0.47", "0.51", "0.58"],
    ["Zone cultivée", "", "", "", "", "", "", "", "", ""],
    ["","Plat 0-2%", "0.31", "0.34", "0.36", "0.4", "", "0.43", "0.47", "0.57"],
    ["","Moyen 2-7%", "0.35", "0.38", "0.41", "0.44", "", "0.48", "0.51", "0.6"],
    ["","Raide >7%", "0.39", "0.42", "0.44", "0.48", "", "0.51", "0.54", "0.61"],
    ["Pâturage, prairie", "", "", "", "", "", "", "", "", ""],
    ["","Plat 0-2%", "0.25", "0.28", "0.3", "0.34", "", "0.37", "0.41", "0.53"],
    ["","Moyen 2-7%", "0.33", "0.36", "0.38", "0.42", "", "0.45", "0.49", "0.58"],
    ["","Raide >7%", "0.37", "0.4", "0.42", "0.46", "", "0.49", "0.53", "0.6"],
    ["Fôret, bois", "", "", "", "", "", "", "", "", ""],
    ["","Plat 0-2%", "0.22", "0.25", "0.28", "0.31", "", "0.35", "0.39", "0.48"],
    ["","Moyen 2-7%", "0.31", "0.34", "0.36", "0.4", "", "0.43", "0.47", "0.56"],
    ["","Raide >7%", "0.35", "0.39", "0.41", "0.45", "", "0.48", "0.52", "0.58"],
]

data_analyzer = DataAnalyzer(rows)
data_analyzer.interpolate_values()

+----------------------+------------+-----------+-----------+------------+--------+------------+------------+-------------+-------------+
|   Type de surface    |   Pente    | T = 2 ans | T = 5 ans | T = 10 ans | T = 25 | T = 30 ans | T = 50 ans | T = 100 ans | T = 500 ans |
+----------------------+------------+-----------+-----------+------------+--------+------------+------------+-------------+-------------+
|       Asphalte       |     -      |    0.73   |    0.77   |    0.81    |  0.86  |    0.89    |    0.9     |     0.95    |      1      |
|      Béton,toit      |     -      |    0.75   |    0.8    |    0.83    |  0.88  |    0.91    |    0.92    |     0.97    |      1      |
|   Zones enherbées    |            |           |           |            |        |            |            |             |             |
|   (pelouse, parc…)   |            |           |           |            |        |            |            |             |             |
|  Conditions faibles  |          

## Entrants ##

In [23]:
basin_info_obj = BasinInfo()
basin_info = basin_info_obj.create_table()

Entrer le nombre des bassins :  2
Le nom du bassin versant 1:  a
La pente du basin 'a' en m/m:  0.2
La Surface Asphalte (ha) du basin 'a':  0.3
La Surface Béton,Toit (ha) du basin 'a':  10
La Surface Zones enherbées (ha) du basin 'a':  10
La Surface Zone cultivée (ha) du basin 'a':  10
La Surface Pâturage, Prairie (ha) du basin 'a':  10
La Surface Fôret (ha) du basin 'a':  10
Le nom du bassin versant 2:  i
La pente du basin 'i' en m/m:  0.3
La Surface Asphalte (ha) du basin 'i':  10
La Surface Béton,Toit (ha) du basin 'i':  10
La Surface Zones enherbées (ha) du basin 'i':  10
La Surface Zone cultivée (ha) du basin 'i':  10
La Surface Pâturage, Prairie (ha) du basin 'i':  10
La Surface Fôret (ha) du basin 'i':  10


+----------------------+-------+-----------------------+-------------------------+------------------------------+----------------------------+--------------------------------+--------------------+
| Nom du basin versant | Pente | Surface Asphalte (ha) | Surface Béton,Toit (ha) | Surface Zones enherbées (ha) | Surface Zone cultivée (ha) | Surface Pâturage, Prairie (ha) | Surface Fôret (ha) |
+----------------------+-------+-----------------------+-------------------------+------------------------------+----------------------------+--------------------------------+--------------------+
|          a           |  0.2  |          0.3          |            10           |              10              |             10             |               10               |         10         |
|          i           |  0.3  |           10          |            10           |              10              |             10             |               10               |         10         |
+--------------

## Estimation des coefficients de ruissellement ##

In [51]:
coefficients = basin_info_obj.calculer_coefficients_ruissellement(basin_info)
table_creator = TableCreator(basin_info, coefficients)
table = table_creator.create_table()
print(table)

+-----------------------+---------------------+-------------+------------------------------+------------------------+----------------+----------------+-----------------+-----------------+-----------------+-----------------+------------------+
| Nom du bassin versant | Surface totale (ha) | Pente (m/m) | Surface imperméabilisée (ha) | Surface naturelle (ha) | CR (T=2 years) | CR (T=5 years) | CR (T=10 years) | CR (T=25 years) | CR (T=30 years) | CR (T=50 years) | CR (T=100 years) |
+-----------------------+---------------------+-------------+------------------------------+------------------------+----------------+----------------+-----------------+-----------------+-----------------+-----------------+------------------+
|           a           |         50.5        |     0.2     |             10.3             |          40.0          |      0.45      |      0.49      |       0.51      |       0.55      |       0.58      |       0.59      |       0.62       |
|           i           |   